# **Réseau de Neurones à partir de zéro pour la Classification du dataset MNIST**

## Importation des bibliothèques

In [34]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import StandardScaler

## Chargement et prétraitement des données

In [35]:
# Chargement des données MNIST
mnist = fetch_openml('mnist_784', version=1)
X = mnist.data / 255.0  # Normalisation des images
y = mnist.target.astype(int)

# Séparation des données d'entraînement et de test
X_train, X_test = X[:60000], X[60000:]
y_train, y_test = y[:60000], y[60000:]


##  Création du modèle de réseau de neurones et optimisation avec Descente de Gradient

In [36]:
#  Création du modèle de réseau de neurones (2 couches)
class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size):
        # Initialisation des poids
        self.W1 = np.random.randn(input_size, hidden_size) * 0.01
        self.b1 = np.zeros((1, hidden_size))
        self.W2 = np.random.randn(hidden_size, output_size) * 0.01
        self.b2 = np.zeros((1, output_size))

    def forward(self, X):
        # Propagation avant
        self.z1 = np.dot(X, self.W1) + self.b1
        self.a1 = np.tanh(self.z1)  # Activation tanh
        self.z2 = np.dot(self.a1, self.W2) + self.b2
        self.a2 = self.softmax(self.z2)  # Activation softmax pour la sortie
        return self.a2

    def softmax(self, z):
        e_z = np.exp(z - np.max(z, axis=1, keepdims=True))
        return e_z / np.sum(e_z, axis=1, keepdims=True)

    def compute_loss(self, y_true, y_pred):
        m = y_true.shape[0]
        log_likelihood = -np.log(y_pred[range(m), y_true])
        loss = np.sum(log_likelihood) / m
        return loss

    def backward(self, X, y_true, learning_rate=0.01):
        m = X.shape[0]

        # Calcul des gradients
        delta2 = self.a2
        delta2[range(m), y_true] -= 1
        delta2 /= m

        dW2 = np.dot(self.a1.T, delta2)
        db2 = np.sum(delta2, axis=0, keepdims=True)

        delta1 = np.dot(delta2, self.W2.T) * (1 - np.square(self.a1))
        dW1 = np.dot(X.T, delta1)
        db1 = np.sum(delta1, axis=0, keepdims=True)

        # Mise à jour des poids et biais
        self.W1 -= learning_rate * dW1
        self.b1 -= learning_rate * db1
        self.W2 -= learning_rate * dW2
        self.b2 -= learning_rate * db2

    def predict(self, X):
        output = self.forward(X)
        return np.argmax(output, axis=1)


#  Fonction d'entraînement du modèle

In [37]:
# Entraînement du modèle
def train(model, X_train, y_train, epochs=10, batch_size=64, learning_rate=0.1):
    for epoch in range(epochs):
        num_batches = len(X_train) // batch_size
        for i in range(num_batches):
            X_batch = X_train[i * batch_size:(i + 1) * batch_size]
            y_batch = y_train[i * batch_size:(i + 1) * batch_size]

            # Propagation avant et arrière
            y_pred = model.forward(X_batch)
            model.backward(X_batch, y_batch, learning_rate)

        # Calcul de la perte pour chaque époque
        y_pred_train = model.forward(X_train)
        loss = model.compute_loss(y_train, y_pred_train)
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss}")


# Fonction de test du modèle

In [38]:
# Test du modèle
def test(model, X_test, y_test):
    y_pred_test = model.predict(X_test)
    accuracy = np.mean(y_pred_test == y_test)
    print(f"Test accuracy: {accuracy * 100:.2f}%")


# Initialisation et entraînement du modèle

In [39]:
# Initialisation du modèle
input_size = 784  # 28x28 pixels
hidden_size = 64
output_size = 10  # 10 classes (0-9)

model = NeuralNetwork(input_size, hidden_size, output_size)

# Entraînement du modèle
train(model, X_train.values, y_train.values, epochs=10, batch_size=64, learning_rate=0.1)


Epoch 1/10, Loss: 0.29214395026136525
Epoch 2/10, Loss: 0.22081237990574976
Epoch 3/10, Loss: 0.1795830711192649
Epoch 4/10, Loss: 0.15263296139325194
Epoch 5/10, Loss: 0.13331351528047491
Epoch 6/10, Loss: 0.11873315967855869
Epoch 7/10, Loss: 0.10728217898513484
Epoch 8/10, Loss: 0.09795688578072521
Epoch 9/10, Loss: 0.09013167813470323
Epoch 10/10, Loss: 0.08340556087178765


# Test du modèle

In [40]:
# Test du modèle
test(model, X_test.values, y_test.values)


Test accuracy: 96.76%


# Création du modèle et optimisation avec momentum

In [41]:
# Création du modèle de réseau de neurones (2 couches) avec momentum
class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size, momentum=0.9):
        # Initialisation des poids
        self.W1 = np.random.randn(input_size, hidden_size) * 0.01
        self.b1 = np.zeros((1, hidden_size))
        self.W2 = np.random.randn(hidden_size, output_size) * 0.01
        self.b2 = np.zeros((1, output_size))

        # Initialisation des vitesses pour momentum
        self.vW1 = np.zeros_like(self.W1)
        self.vb1 = np.zeros_like(self.b1)
        self.vW2 = np.zeros_like(self.W2)
        self.vb2 = np.zeros_like(self.b2)

        # Paramètre de momentum
        self.momentum = momentum

    def forward(self, X):
        # Propagation avant
        self.z1 = np.dot(X, self.W1) + self.b1
        self.a1 = np.tanh(self.z1)  # Activation tanh
        self.z2 = np.dot(self.a1, self.W2) + self.b2
        self.a2 = self.softmax(self.z2)  # Activation softmax pour la sortie
        return self.a2

    def softmax(self, z):
        e_z = np.exp(z - np.max(z, axis=1, keepdims=True))
        return e_z / np.sum(e_z, axis=1, keepdims=True)

    def compute_loss(self, y_true, y_pred):
        m = y_true.shape[0]
        log_likelihood = -np.log(y_pred[range(m), y_true])
        loss = np.sum(log_likelihood) / m
        return loss

    def backward(self, X, y_true, learning_rate=0.01):
        m = X.shape[0]

        # Calcul des gradients
        delta2 = self.a2
        delta2[range(m), y_true] -= 1
        delta2 /= m

        dW2 = np.dot(self.a1.T, delta2)
        db2 = np.sum(delta2, axis=0, keepdims=True)

        delta1 = np.dot(delta2, self.W2.T) * (1 - np.square(self.a1))
        dW1 = np.dot(X.T, delta1)
        db1 = np.sum(delta1, axis=0, keepdims=True)

        # Mise à jour avec momentum
        self.vW1 = self.momentum * self.vW1 - learning_rate * dW1
        self.vb1 = self.momentum * self.vb1 - learning_rate * db1
        self.vW2 = self.momentum * self.vW2 - learning_rate * dW2
        self.vb2 = self.momentum * self.vb2 - learning_rate * db2

        # Mise à jour des poids et biais avec la vitesse
        self.W1 += self.vW1
        self.b1 += self.vb1
        self.W2 += self.vW2
        self.b2 += self.vb2

    def predict(self, X):
        output = self.forward(X)
        return np.argmax(output, axis=1)


## Fonction d'entraînement du modèle avec momentum

In [43]:
# Entraînement du modèle avec momentum
def train(model, X_train, y_train, epochs=10, batch_size=64, learning_rate=0.1):
    for epoch in range(epochs):
        num_batches = len(X_train) // batch_size
        for i in range(num_batches):
            X_batch = X_train[i * batch_size:(i + 1) * batch_size]
            y_batch = y_train[i * batch_size:(i + 1) * batch_size]

            # Propagation avant et arrière
            y_pred = model.forward(X_batch)
            model.backward(X_batch, y_batch, learning_rate)

        # Calcul de la perte pour chaque époque
        y_pred_train = model.forward(X_train)
        loss = model.compute_loss(y_train, y_pred_train)
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss}")


# Initialisation et entraînement du modèle avec momentum

In [44]:
# Initialisation du modèle avec momentum
input_size = 784  # 28x28 pixels
hidden_size = 64
output_size = 10  # 10 classes (0-9)

model = NeuralNetwork(input_size, hidden_size, output_size, momentum=0.9)

# Entraînement du modèle
train(model, X_train.values, y_train.values, epochs=10, batch_size=64, learning_rate=0.1)


Epoch 1/10, Loss: 0.15206974050626443
Epoch 2/10, Loss: 0.11782897032571822
Epoch 3/10, Loss: 0.09512212319330719
Epoch 4/10, Loss: 0.08180251355546858
Epoch 5/10, Loss: 0.08717723657438031
Epoch 6/10, Loss: 0.0780436682293898
Epoch 7/10, Loss: 0.06621340438758468
Epoch 8/10, Loss: 0.05662482909482049
Epoch 9/10, Loss: 0.05972824059203405
Epoch 10/10, Loss: 0.05230104274669498


# Test du modèle

In [45]:
# Test du modèle
test(model, X_test.values, y_test.values)


Test accuracy: 96.50%


## Création du modèle et optimisation avec Adagrad

In [46]:
# Création du modèle de réseau de neurones (2 couches) avec Adagrad
class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size, epsilon=1e-8):
        # Initialisation des poids
        self.W1 = np.random.randn(input_size, hidden_size) * 0.01
        self.b1 = np.zeros((1, hidden_size))
        self.W2 = np.random.randn(hidden_size, output_size) * 0.01
        self.b2 = np.zeros((1, output_size))

        # Initialisation des accumulateurs de gradients (pour Adagrad)
        self.GW1 = np.zeros_like(self.W1)
        self.Gb1 = np.zeros_like(self.b1)
        self.GW2 = np.zeros_like(self.W2)
        self.Gb2 = np.zeros_like(self.b2)

        # Paramètre epsilon pour éviter la division par zéro
        self.epsilon = epsilon

    def forward(self, X):
        # Propagation avant
        self.z1 = np.dot(X, self.W1) + self.b1
        self.a1 = np.tanh(self.z1)  # Activation tanh
        self.z2 = np.dot(self.a1, self.W2) + self.b2
        self.a2 = self.softmax(self.z2)  # Activation softmax pour la sortie
        return self.a2

    def softmax(self, z):
        e_z = np.exp(z - np.max(z, axis=1, keepdims=True))
        return e_z / np.sum(e_z, axis=1, keepdims=True)

    def compute_loss(self, y_true, y_pred):
        m = y_true.shape[0]
        log_likelihood = -np.log(y_pred[range(m), y_true])
        loss = np.sum(log_likelihood) / m
        return loss

    def backward(self, X, y_true, learning_rate=0.01):
        m = X.shape[0]

        # Calcul des gradients
        delta2 = self.a2
        delta2[range(m), y_true] -= 1
        delta2 /= m

        dW2 = np.dot(self.a1.T, delta2)
        db2 = np.sum(delta2, axis=0, keepdims=True)

        delta1 = np.dot(delta2, self.W2.T) * (1 - np.square(self.a1))
        dW1 = np.dot(X.T, delta1)
        db1 = np.sum(delta1, axis=0, keepdims=True)

        # Mise à jour des accumulateurs pour Adagrad
        self.GW1 += dW1**2
        self.Gb1 += db1**2
        self.GW2 += dW2**2
        self.Gb2 += db2**2

        # Mise à jour des poids et biais avec Adagrad
        self.W1 -= learning_rate * dW1 / (np.sqrt(self.GW1) + self.epsilon)
        self.b1 -= learning_rate * db1 / (np.sqrt(self.Gb1) + self.epsilon)
        self.W2 -= learning_rate * dW2 / (np.sqrt(self.GW2) + self.epsilon)
        self.b2 -= learning_rate * db2 / (np.sqrt(self.Gb2) + self.epsilon)

    def predict(self, X):
        output = self.forward(X)
        return np.argmax(output, axis=1)


## Initialisation et entraînement du modèle avec Adagrad

In [55]:
# Initialisation du modèle avec Adagrad
input_size = 784  # 28x28 pixels
hidden_size = 64
output_size = 10  # 10 classes (0-9)

model = NeuralNetwork(input_size, hidden_size, output_size)

# Entraînement du modèle
train(model, X_train.values, y_train.values, epochs=10, batch_size=64, learning_rate=0.1)


Epoch 1/10, Loss: 0.8900416888062903
Epoch 2/10, Loss: 0.876431567826437
Epoch 3/10, Loss: 0.8715988735979321
Epoch 4/10, Loss: 0.8697767930073292
Epoch 5/10, Loss: 0.8689500252623208
Epoch 6/10, Loss: 0.8684823789709594
Epoch 7/10, Loss: 0.868148706843655
Epoch 8/10, Loss: 0.8679259948085648
Epoch 9/10, Loss: 0.8677811178014015
Epoch 10/10, Loss: 0.8676088639955631


## Test du modèle

In [51]:
# Test du modèle
test(model, X_test.values, y_test.values)


Test accuracy: 96.60%


## Implémentation du réseau de neurones avec mise à jour des poids via Proximal Stochastic Approximation (PSA)

In [52]:
# Création du modèle de réseau de neurones (2 couches)
class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size, lambda_psa=0.1):
        # Initialisation des poids
        self.W1 = np.random.randn(input_size, hidden_size) * 0.01
        self.b1 = np.zeros((1, hidden_size))
        self.W2 = np.random.randn(hidden_size, output_size) * 0.01
        self.b2 = np.zeros((1, output_size))
        self.lambda_psa = lambda_psa  # Paramètre de régularisation PSA

    def forward(self, X):
        # Propagation avant
        self.z1 = np.dot(X, self.W1) + self.b1
        self.a1 = np.tanh(self.z1)  # Activation tanh
        self.z2 = np.dot(self.a1, self.W2) + self.b2
        self.a2 = self.softmax(self.z2)  # Activation softmax pour la sortie
        return self.a2

    def softmax(self, z):
        e_z = np.exp(z - np.max(z, axis=1, keepdims=True))
        return e_z / np.sum(e_z, axis=1, keepdims=True)

    def compute_loss(self, y_true, y_pred):
        m = y_true.shape[0]
        log_likelihood = -np.log(y_pred[range(m), y_true])
        loss = np.sum(log_likelihood) / m
        return loss

    def backward(self, X, y_true, learning_rate=0.01):
        m = X.shape[0]

        # Calcul des gradients
        delta2 = self.a2
        delta2[range(m), y_true] -= 1
        delta2 /= m

        dW2 = np.dot(self.a1.T, delta2)
        db2 = np.sum(delta2, axis=0, keepdims=True)

        delta1 = np.dot(delta2, self.W2.T) * (1 - np.square(self.a1))
        dW1 = np.dot(X.T, delta1)
        db1 = np.sum(delta1, axis=0, keepdims=True)

        # PSA update: appliquer un terme de régularisation sur les poids (Proximal)
        self.W1 -= learning_rate * (dW1 + self.lambda_psa * self.W1)  # Proximal pour W1
        self.b1 -= learning_rate * db1
        self.W2 -= learning_rate * (dW2 + self.lambda_psa * self.W2)  # Proximal pour W2
        self.b2 -= learning_rate * db2

    def predict(self, X):
        output = self.forward(X)
        return np.argmax(output, axis=1)


## Initialisation et entraînement du modèle avec PSA

In [53]:
# Initialisation du modèle
input_size = 784  # 28x28 pixels
hidden_size = 64
output_size = 10  # 10 classes (0-9)
lambda_psa = 0.1  # Paramètre de régularisation PSA

model = NeuralNetwork(input_size, hidden_size, output_size, lambda_psa)

# Entraînement du modèle
train(model, X_train.values, y_train.values, epochs=10, batch_size=64, learning_rate=0.1)


Epoch 1/10, Loss: 0.8861966979405707
Epoch 2/10, Loss: 0.874387282161788
Epoch 3/10, Loss: 0.8713493574683842
Epoch 4/10, Loss: 0.8703042617739509
Epoch 5/10, Loss: 0.8695128386812778
Epoch 6/10, Loss: 0.8687448810588737
Epoch 7/10, Loss: 0.8680075390808283
Epoch 8/10, Loss: 0.8675548168972784
Epoch 9/10, Loss: 0.8672945959214289
Epoch 10/10, Loss: 0.8671524854892657


## Test du modèle

In [54]:
# Test du modèle
test(model, X_test.values, y_test.values)

Test accuracy: 84.21%
